## Import package

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

## Load dataset: Train

In [2]:
train = pd.read_csv("../../../../data/train/train.csv")
test = pd.read_csv("../../../../data/test/test.csv")

In [3]:
print(train.shape)
print(test.shape)

(9557, 143)
(23856, 142)


In [4]:
train.head(5)

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target
0,ID_279628684,190000.0,0,3,0,1,1,0,NaN,0,...,100,1849,1,100,0,1.000000,0.0,100.0,1849,4
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,144,4489,1,144,0,1.000000,64.0,144.0,4489,4
2,ID_68de51c94,NaN,0,8,0,1,1,0,NaN,0,...,121,8464,1,0,0,0.250000,64.0,121.0,8464,4
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,81,289,16,121,4,1.777778,1.0,121.0,289,4
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,121,1369,16,121,4,1.777778,1.0,121.0,1369,4


In [5]:
test.head(5)

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,age,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq
0,ID_2f6873615,NaN,0,5,0,1,1,0,NaN,1,...,4,0,16,9,0,1,2.25,0.25,272.25,16
1,ID_1c78846d2,NaN,0,5,0,1,1,0,NaN,1,...,41,256,1681,9,0,1,2.25,0.25,272.25,1681
2,ID_e5442cf6a,NaN,0,5,0,1,1,0,NaN,1,...,41,289,1681,9,0,1,2.25,0.25,272.25,1681
3,ID_a8db26a79,NaN,0,14,0,1,1,1,1.0,0,...,59,256,3481,1,256,0,1.00,0.00,256.00,3481
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,18,121,324,1,0,1,0.25,64.00,NaN,324


In [6]:
f = open("../../resources/continuousFeatures", "r")
continuous_columns = f.read().split(",")
f.close()

f = open("../../resources/categoricalFeatures", "r")
categorical_columns = f.read().split(",")
f.close()

print("Number of numerical columns: {0}".format(len(continuous_columns)))
print("Number of categorical columns: {0}".format(len(categorical_columns)))

Number of numerical columns: 38
Number of categorical columns: 102


In [7]:
y_train = train["Target"]
print(y_train.shape)

(9557,)


In [8]:
X_train = train[categorical_columns]
X_test = test[categorical_columns]
print(X_train.shape)
print(X_test.shape)

(9557, 102)
(23856, 102)


## Classification: KNeighborsClassifier

In [9]:
nearest_neighbors = KNeighborsClassifier()

In [10]:
model = nearest_neighbors.fit(X_train, y_train)

In [20]:
prediction_train = model.predict(X_train)
prediction_test = model.predict(X_test)

In [21]:
f1_score(y_train, prediction_train, average=None)  

array([ 0.77424023,  0.76852459,  0.70423847,  0.90993219])

In [22]:
accuracy_score(y_train, prediction_train)

0.85445223396463321

In [23]:
unique_elements, counts_elements = np.unique(prediction_train, return_counts=True)
print(unique_elements)
print(counts_elements)

[1 2 3 4]
[ 627 1453  938 6539]


In [24]:
unique_elements, counts_elements = np.unique(prediction_test, return_counts=True)
print(unique_elements)
print(counts_elements)

[1 2 3 4]
[ 1396  3239  1945 17276]


In [25]:
print(type(prediction_test))
print(type(test["Id"]))

d = {"Id": test["Id"], "Target": prediction_test}
submission = pd.DataFrame(d)
submission.head()

submission.to_csv("../../../../submission/sklearn/kNearestNeighbors/01.csv", index=False)

<type 'numpy.ndarray'>
<class 'pandas.core.series.Series'>


## Hyper-parameters tuning

In [39]:
# param_grid = [{"n_neighbors": [5, 10, 15, 20]}]

param_grid = [{"n_neighbors": [3, 5, 7, 9],
             "weights": ["uniform", "distance"],
             "algorithm": ["ball_tree", "kd_tree", "brute"],
             "leaf_size": [10, 20, 30],
             "p": [1, 2]
             }]

In [40]:
grid = GridSearchCV(nearest_neighbors, cv=3, param_grid=param_grid, verbose=1)

In [ ]:
grid.fit(X_train, y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


In [ ]:
grid.best_params_

In [ ]:
grid.best_index_

In [ ]:
grid.best_score_

In [ ]:
grid.cv_results_

In [ ]:
prediction_train = grid.predict(X_train)

In [ ]:
unique_elements, counts_elements = np.unique(prediction_train, return_counts=True)
print(unique_elements)
print(counts_elements)

In [ ]:
unique_elements, counts_elements = np.unique(grid.predict(X_test), return_counts=True)
print(unique_elements)
print(counts_elements)

In [ ]:
prediction_test = grid.predict(X_test)

In [ ]:
print(type(prediction_test))
print(type(test["Id"]))

d = {"Id": test["Id"], "Target": prediction_test}
submission = pd.DataFrame(d)
submission.head()

submission.to_csv("../../../../submission/sklearn/kNearestNeighbors/02.csv", index=False)